In [2]:
#yield from as a return value
def accumulate():
	tally = 0
	while 1:
		next = yield 1
		if next == None:
			return tally # terminate the generator
		tally += next

def gather_tallies(tallies):
	while 1:
		res = yield from accumulate() # yield from get "return value", send get "yield value"
		tallies.append(res)

tallies = []
g = gather_tallies(tallies)

g.send(None) #initialize, make generator prepare to receive first parameter
for i in range(4):
	g.send(i)

g.send(None) #finish first generator
for i in range(5):
	g.send(i)
g.send(None) #finish second generator
print(tallies)

[6, 10]


In [3]:
def consumer():
	r = ''
	print('first consumer execution. 11111111111111111')
	while True:
		n = yield r # receive n from send. But yield value immediately when arriving at the block
		print('After yield')
		if not n:
			return # terminate the generator
		print('[Consumer] consumes %s...' % n)
		r = '200 OK'

def produce(c):
	c.send(None) # make consumer block at n = yield r, when r is ''
	print('first produce execution')
	n = 0
	while n < 5:
		n = n + 1
		print('[Producer] produces %s...' %n)
		r = c.send(n)
		print('[Producer] consumer return: %s' %r)
	c.close()

c = consumer()
produce(c)

first consumer execution. 11111111111111111
first produce execution
[Producer] produces 1...
After yield
[Consumer] consumes 1...
[Producer] consumer return: 200 OK
[Producer] produces 2...
After yield
[Consumer] consumes 2...
[Producer] consumer return: 200 OK
[Producer] produces 3...
After yield
[Consumer] consumes 3...
[Producer] consumer return: 200 OK
[Producer] produces 4...
After yield
[Consumer] consumes 4...
[Producer] consumer return: 200 OK
[Producer] produces 5...
After yield
[Consumer] consumes 5...
[Producer] consumer return: 200 OK


In [4]:
import threading
import asyncio

@asyncio.coroutine
def hello(id):
    print('Hello world! (%s) %s' % (threading.currentThread(), id))
    yield from asyncio.sleep(1)
    print('Hello again! (%s) %s' % (threading.currentThread(), id))

loop = asyncio.get_event_loop()
tasks = [hello(1), hello(2)]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world! (<_MainThread(MainThread, started 175496)>) 2
Hello world! (<_MainThread(MainThread, started 175496)>) 1
Hello again! (<_MainThread(MainThread, started 175496)>) 2
Hello again! (<_MainThread(MainThread, started 175496)>) 1


In [8]:
# remember to reset kernel in Ipython notebook
import threading 
import asyncio

async def hello(id):
    print('Hello world! (%s) %s' % (threading.currentThread(), id))
    await asyncio.sleep(1)
    print('Hello again! (%s) %s' % (threading.currentThread(), id))

asyncio.set_event_loop(asyncio.new_event_loop()) # If after close event loop above
loop = asyncio.get_event_loop()
tasks = [hello(1), hello(2)] #sequence random
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world! (<_MainThread(MainThread, started 35472)>) 2
Hello world! (<_MainThread(MainThread, started 35472)>) 1
Hello again! (<_MainThread(MainThread, started 35472)>) 2
Hello again! (<_MainThread(MainThread, started 35472)>) 1


In [11]:
import asyncio,threading
async def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = await connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    await writer.drain()
    while True:
        line = await reader.readline()
        await asyncio.sleep(1)
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

asyncio.set_event_loop(asyncio.new_event_loop())
loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.163.com...
wget www.sina.com.cn...
wget www.sohu.com...
www.163.com header > HTTP/1.0 302 Moved Temporarily
www.sina.com.cn header > HTTP/1.1 200 OK
www.sohu.com header > HTTP/1.1 200 OK
www.163.com header > Server: Cdn Cache Server V2.0
www.sina.com.cn header > Date: Fri, 19 May 2017 00:58:46 GMT
www.sohu.com header > Content-Type: text/html;charset=utf-8
www.163.com header > Date: Fri, 19 May 2017 00:58:46 GMT
www.sina.com.cn header > Server: PWS/8.2.0.7
www.sohu.com header > Content-Length: 133236
www.163.com header > Content-Length: 0
www.sina.com.cn header > X-Px: ht h0-s1025.p1-iad.cdngp.net
www.sohu.com header > Connection: close
www.163.com header > Location: http://www.163.com/special/0077jt/error_isp.html
www.sina.com.cn header > Cache-Control: max-age=3600
www.sohu.com header > Server: nginx
www.163.com header > Connection: close
www.sina.com.cn header > Expires: Fri, 19 May 2017 01:44:14 GMT
www.sohu.com header > Date: Fri, 19 May 2017 00:58:44 GMT
www.sina.com.cn 

In [ ]:
import asyncio
from aiohttp import web

async def index(request):
	await asyncio.sleep(0.5)
	return web.Response(body=b'<h1>Index</h1>', content_type = 'text/html')

async def hello(request):
	await asyncio.sleep(0.5)
	text = '<h1>hello, %s!</h1>' % request.match_info['name']
	return web.Response(body=text.encode('utf-8'), content_type = 'text/html')

async def init(loop):
	app = web.Application(loop=loop)
	app.router.add_route('GET', '/', index)
	# Handler is an any callable that accepts a single Request argument and returns a StreamResponse derived (e.g. Response) instance.
	app.router.add_route('GET', '/hello/{name}', hello)
	srv = await loop.create_server(app.make_handler(), '127.0.0.1', 8001)
	print('serving on', srv.sockets[0].getsockname())
	return srv

asyncio.set_event_loop(asyncio.new_event_loop())
loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()

serving on ('127.0.0.1', 8001)
